In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import cv2
from PIL import Image
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix 
from sklearn.metrics import f1_score
from sklearn.linear_model import SGDClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split
import random

In [2]:
my_data=pd.read_csv('../data/image_data.csv')
data_array=my_data.to_numpy()
images_path='../data/pocus_images/convex/'
images_data=[]
for i in data_array:
    img = cv2.imread(images_path+i[0]+'.'+i[2], cv2.IMREAD_GRAYSCALE)
    img=cv2.equalizeHist(img)
    img=cv2.resize(img, (224,224), interpolation = cv2.INTER_AREA)
    img_1d=img.reshape(224*224)
    images_data.append(img_1d)


In [32]:
number_of_models=21
X=np.asarray(images_data)
y=np.asarray(my_data['Label'])
Xandy=[]
models=[]
#creating models
for i in range(number_of_models):
    models.append(SGDClassifier(random_state=42))

#Creating array test splits for models
#X_train, X_test, y_train, y_test 
for i in range(number_of_models):
    x = train_test_split(X, y, test_size=0.2, random_state=42)
    Xandy.append(x)

In [33]:
#Equalizing Data
for i in range(number_of_models):
    x=Xandy[i][0]
    y=Xandy[i][2]
    cov=np.count_nonzero(y == 'covid')
    pneu=np.count_nonzero(y == 'pneumonia')
    reg=np.count_nonzero(y == 'regular')
    #deleting random pneumonia elements from each model set
    while(cov!=pneu):
        index=random.randint(0, len(x)-1)
        if(y[index]=='pneumonia'):
            x=np.delete(x,index,0)
            y=np.delete(y,index,0)
            pneu-=1
    #deleting random regular elements from each model set       
    while(cov!=reg):
        index=random.randint(0, len(x)-1)
        if(y[index]=='regular'):
            x=np.delete(x,index,0)
            y=np.delete(y,index,0)
            reg-=1
    Xandy[i][0]=x
    Xandy[i][2]=y

In [34]:

print(np.count_nonzero(Xandy[2][2] == 'pneumonia'))

26


In [35]:
#Train each Model
for i in range(number_of_models):
    print(models[i].fit(Xandy[i][0],Xandy[i][2]))

SGDClassifier(random_state=42)
SGDClassifier(random_state=42)
SGDClassifier(random_state=42)
SGDClassifier(random_state=42)
SGDClassifier(random_state=42)
SGDClassifier(random_state=42)
SGDClassifier(random_state=42)
SGDClassifier(random_state=42)
SGDClassifier(random_state=42)
SGDClassifier(random_state=42)
SGDClassifier(random_state=42)
SGDClassifier(random_state=42)
SGDClassifier(random_state=42)
SGDClassifier(random_state=42)
SGDClassifier(random_state=42)
SGDClassifier(random_state=42)
SGDClassifier(random_state=42)
SGDClassifier(random_state=42)
SGDClassifier(random_state=42)
SGDClassifier(random_state=42)
SGDClassifier(random_state=42)


In [36]:
#Get predicted value from xtest of each model 
predicted=[]
for i in range(number_of_models):
    predicted.append(models[i].predict(Xandy[i][1]))
#First model prediction
predicted[0]

array(['pneumonia', 'covid', 'pneumonia', 'pneumonia', 'regular', 'covid',
       'pneumonia', 'pneumonia', 'pneumonia', 'covid', 'covid', 'covid',
       'pneumonia', 'covid', 'covid', 'regular', 'pneumonia', 'regular',
       'pneumonia', 'pneumonia', 'covid', 'covid', 'covid', 'regular',
       'regular', 'regular', 'regular', 'covid', 'regular', 'covid',
       'covid'], dtype='<U9')

In [37]:
#Method to get most predicted element
from mostCommon import most_common

In [38]:
#Create 1 predict array by model voting
voted_prediction=[]
for i in range(len(predicted[0])):
    voting=[]
    for j in range(len(predicted)):
        voting.append(predicted[j][i])
    voted_prediction.append(most_common(voting))
#this is the final voted prediction
print(voted_prediction)

['pneumonia', 'covid', 'pneumonia', 'pneumonia', 'regular', 'covid', 'pneumonia', 'pneumonia', 'pneumonia', 'covid', 'covid', 'covid', 'pneumonia', 'covid', 'covid', 'regular', 'pneumonia', 'regular', 'pneumonia', 'regular', 'covid', 'covid', 'covid', 'regular', 'regular', 'regular', 'regular', 'covid', 'regular', 'covid', 'covid']


In [39]:
#This is the y_test which is the same for all models
print(Xandy[0][3])

['pneumonia' 'covid' 'pneumonia' 'pneumonia' 'regular' 'covid' 'pneumonia'
 'pneumonia' 'pneumonia' 'regular' 'pneumonia' 'covid' 'pneumonia' 'covid'
 'covid' 'regular' 'regular' 'regular' 'pneumonia' 'regular' 'covid'
 'pneumonia' 'covid' 'regular' 'regular' 'regular' 'regular' 'pneumonia'
 'regular' 'covid' 'covid']


In [40]:
conf_matrix=confusion_matrix(Xandy[0][3],voted_prediction)
conf_matrix

array([[9, 0, 0],
       [3, 8, 0],
       [1, 1, 9]], dtype=int64)

In [41]:
print(metrics.classification_report(Xandy[0][3],voted_prediction, digits=3))

              precision    recall  f1-score   support

       covid      0.692     1.000     0.818         9
   pneumonia      0.889     0.727     0.800        11
     regular      1.000     0.818     0.900        11

    accuracy                          0.839        31
   macro avg      0.860     0.848     0.839        31
weighted avg      0.871     0.839     0.841        31

